In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

/home/adityajakka/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 7
batch_size = 16
nb_epoch = 25

In [3]:
from keras.models import Model
model = VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
model.layers.pop()
model.layers.pop()
model.layers.pop()
model.layers.pop()

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
for layer in model.layers[7:]:
    layer.trainable = False

In [8]:
classifier = Sequential()

In [9]:
classifier.add(model)

In [10]:
classifier.add(Flatten())

In [11]:
classifier.add(Dense(1024, activation='relu'))
classifier.add(Dropout(0.3))

In [12]:
classifier.add(Dense(7, activation='sigmoid'))

In [13]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


In [14]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         7635264   
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 7175      
Total params: 33,333,575
Trainable params: 25,958,471
Non-trainable params: 7,375,104
_________________________________________________________________


In [15]:
early = EarlyStopping(monitor='val_acc', patience=5, mode='auto')

In [16]:
train = np.load('train_mat.npy')

In [17]:
test = np.load('test_mat.npy')

In [18]:
X_train = train[:,-1]
X_train = np.stack(X_train, axis=0)

In [19]:
X_train = preprocess_input(X_train)

In [20]:
y_train = train[:,1:-1]

In [21]:
X_test = test[:,-1]
X_test = np.stack(X_test, axis=0)
X_test = preprocess_input(X_test)

In [22]:
datagen.fit(X_train)

In [23]:
y_test = test[:,1:-1]

from keras.optimizers import SGD
sgd = SGD(lr=2e-3, decay=1e-6, momentum=0.9, nesterov=True)

In [24]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy','accuracy'])

In [28]:
classifier.fit_generator(datagen.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train), epochs=12,  validation_data = (X_test, y_test), callbacks = [early])

Epoch 1/12
2078/2078 [==============================] - 1465s 705ms/step - loss: 0.4473 - categorical_accuracy: 0.4858 - acc: 0.8003 - val_loss: 3.0341 - val_categorical_accuracy: 0.4385 - val_acc: 0.7668
Epoch 2/12
2078/2078 [==============================] - 1461s 703ms/step - loss: 0.3940 - categorical_accuracy: 0.5403 - acc: 0.8244 - val_loss: 3.3253 - val_categorical_accuracy: 0.4596 - val_acc: 0.7698
Epoch 3/12
2078/2078 [==============================] - 1460s 703ms/step - loss: 0.3465 - categorical_accuracy: 0.5816 - acc: 0.8449 - val_loss: 3.2961 - val_categorical_accuracy: 0.4923 - val_acc: 0.7706
Epoch 4/12
2078/2078 [==============================] - 1461s 703ms/step - loss: 0.3106 - categorical_accuracy: 0.6106 - acc: 0.8613 - val_loss: 3.4361 - val_categorical_accuracy: 0.4942 - val_acc: 0.7599
Epoch 5/12
2078/2078 [==============================] - 1461s 703ms/step - loss: 0.2806 - categorical_accuracy: 0.6373 - acc: 0.8750 - val_loss: 3.4310 - val_categorical_accuracy: 

In [25]:
y_pred = model.predict(X_test)

In [26]:
y_pred_1 = y_pred > 0.50 #try 0.4

In [27]:
count_var = 0
for i in range(len(y_test)):
    if np.array_equal(y_test[i], y_pred_1[i]) == True:
        count_var += 1
    else:
        count_var = count_var      

In [28]:
count_var/len(y_test)

0.0

In [47]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [48]:
hamming_score(y_test, y_pred_1, normalize=True, sample_weight=None)

0.31566868505644013